In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import pandas as pd
import datetime as dt


In [2]:
PROJECT = 'ballosodeuk'
client = bigquery.Client(project=PROJECT)

In [9]:
def estimate_query_cost(client, query):
    job_config = bigquery.QueryJobConfig(
        dry_run=True,
        use_query_cache=False
    )
    query_job = client.query(query, job_config=job_config)
    bytes_processed = query_job.total_bytes_processed
    kb = round(bytes_processed / 1024, 2)
    gb = round(bytes_processed / (1024 ** 3), 2)
    print("예상 쿼리 용량: {} KB, ({} GB)".format(kb, gb))

In [5]:
def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

In [8]:
query_name = "0718_유저LTV_생성"
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()

estimate_query_cost(client,query)

예상 쿼리 용량: 668427.09 KB, (0.64 GB)


In [9]:
run_query(client,query).to_dataframe()

In [53]:

df = run_query(client,query).to_dataframe()
df

,subParam,gmv,commission
0,1eced8e0-0d34-467a-b56b-3b64703bf371,21540.0,970.0
1,4a20e7e0-1429-4f58-bdb2-a549634e8d40,11590.0,523.0
2,b4d35085-865c-409d-afd7-107f2ff2cc19,55420.0,2495.0
3,b70d1ecc-2310-413b-aa0a-c6240368b9f6,0.0,0.0
4,bcfc7195-e991-4646-b2b7-20ecb66beba7,24960.0,1124.0
...,...,...,...
852,4a628023-253c-4b76-b820-bd027beaf603,49760.0,2240.0
853,82985081-dff3-4f56-8938-8d5490913185,25420.0,1144.0
854,acff2a95-0fa7-4a4a-979a-50df518c1c6a,110330.0,4967.0
855,b2e689e8-82c4-4f10-a587-590c2cb3841b,12960.0,584.0


In [45]:
df.query("subParam.isna()")

,subParam,gmv,commission


In [49]:
df.loc[857]

subParam                
gmv           41030359.0
commission     1209165.0
Name: 857, dtype: object

임시

In [3]:
query = """ 
SELECT
  i.Is_First_Event_per_User_ID, i.Is_First_Event_per_Device_ID,
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, Event_Category, 
  count(distinct Airbridge_Device_ID) as abid, count(distinct User_ID) as wkid
 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30"
  and Event_Category in ("Install (App)", "Sign-in (App)")
 group by year, month, Event_Category, i.Is_First_Event_per_User_ID, i.Is_First_Event_per_Device_ID
"""

In [4]:
def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,Is_First_Event_per_User_ID,Is_First_Event_per_Device_ID,year,month,Event_Category,abid,wkid
0,<NA>,True,2024,6,Install (App),88825,0
1,<NA>,False,2024,6,Install (App),30698,0
2,True,True,2024,6,Install (App),741,741
3,False,False,2024,6,Install (App),619,583
4,False,True,2024,6,Install (App),171,169
5,True,False,2024,6,Install (App),226,226
6,False,False,2024,6,Sign-in (App),507488,498910
7,True,True,2024,6,Sign-in (App),41258,41255
8,False,True,2024,6,Sign-in (App),6542,6085
9,True,False,2024,6,Sign-in (App),1958,1972


In [5]:
df.query("Event_Category=='Sign-in (App)'").abid.sum()

557333

In [19]:
query = """ 
SELECT
  i.Is_First_Event_per_User_ID,
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, Event_Category, 
  count(distinct Airbridge_Device_ID) as abid, count(distinct User_ID) as wkid
 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30"
  and Event_Category in ("Install (App)", "Sign-in (App)")
 group by year, month, Event_Category, i.Is_First_Event_per_User_ID
"""

def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,Is_First_Event_per_User_ID,year,month,Event_Category,abid,wkid
0,<NA>,2024,6,Install (App),118049,0
1,False,2024,6,Install (App),789,745
2,True,2024,6,Install (App),967,967
3,False,2024,6,Sign-in (App),510100,500236
4,True,2024,6,Sign-in (App),43188,43227
5,<NA>,2024,6,Sign-in (App),87,0


In [18]:
df.query("Event_Category=='Sign-in (App)'").abid.sum(),df.query("Event_Category=='Sign-in (App)'").wkid.sum()

(556797, 547602)

In [20]:
query = """ 
SELECT
  i.Is_First_Event_per_Device_ID,
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, Event_Category, 
  count(distinct Airbridge_Device_ID) as abid, count(distinct User_ID) as wkid
 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30"
  and Event_Category in ("Install (App)", "Sign-in (App)")
 group by year, month, Event_Category, i.Is_First_Event_per_Device_ID
"""

def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,Is_First_Event_per_Device_ID,year,month,Event_Category,abid,wkid
0,True,2024,6,Install (App),89737,910
1,False,2024,6,Install (App),31522,808
2,False,2024,6,Sign-in (App),508964,500398
3,True,2024,6,Sign-in (App),47833,47204


In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import pandas as pd
import datetime as dt


In [ ]:
PROJECT = 'ballosodeuk'
client = bigquery.Client(project=PROJECT)

In [ ]:
def estimate_query_cost(client, query):
    job_config = bigquery.QueryJobConfig(
        dry_run=True,
        use_query_cache=False
    )
    query_job = client.query(query, job_config=job_config)
    bytes_processed = query_job.total_bytes_processed
    kb = round(bytes_processed / 1024, 2)
    gb = round(bytes_processed / (1024 ** 3), 2)
    print("예상 쿼리 용량: {} KB, ({} GB)".format(kb, gb))

In [ ]:
def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

In [ ]:
query_name = "0718_유저LTV_생성"
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()

estimate_query_cost(client,query)

예상 쿼리 용량: 668427.09 KB, (0.64 GB)


In [ ]:
run_query(client,query).to_dataframe()

In [ ]:

df = run_query(client,query).to_dataframe()
df

,subParam,gmv,commission
0,1eced8e0-0d34-467a-b56b-3b64703bf371,21540.0,970.0
1,4a20e7e0-1429-4f58-bdb2-a549634e8d40,11590.0,523.0
2,b4d35085-865c-409d-afd7-107f2ff2cc19,55420.0,2495.0
3,b70d1ecc-2310-413b-aa0a-c6240368b9f6,0.0,0.0
4,bcfc7195-e991-4646-b2b7-20ecb66beba7,24960.0,1124.0
...,...,...,...
852,4a628023-253c-4b76-b820-bd027beaf603,49760.0,2240.0
853,82985081-dff3-4f56-8938-8d5490913185,25420.0,1144.0
854,acff2a95-0fa7-4a4a-979a-50df518c1c6a,110330.0,4967.0
855,b2e689e8-82c4-4f10-a587-590c2cb3841b,12960.0,584.0


In [ ]:
df.query("subParam.isna()")

,subParam,gmv,commission


In [ ]:
df.loc[857]

subParam                
gmv           41030359.0
commission     1209165.0
Name: 857, dtype: object

임시

In [ ]:
query = """ 
SELECT
  i.Is_First_Event_per_User_ID, i.Is_First_Event_per_Device_ID,
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, Event_Category, 
  count(distinct Airbridge_Device_ID) as abid, count(distinct User_ID) as wkid
 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30"
  and Event_Category in ("Install (App)", "Sign-in (App)")
 group by year, month, Event_Category, i.Is_First_Event_per_User_ID, i.Is_First_Event_per_Device_ID
"""

In [ ]:
def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,Is_First_Event_per_User_ID,Is_First_Event_per_Device_ID,year,month,Event_Category,abid,wkid
0,<NA>,True,2024,6,Install (App),88825,0
1,<NA>,False,2024,6,Install (App),30698,0
2,True,True,2024,6,Install (App),741,741
3,False,False,2024,6,Install (App),619,583
4,False,True,2024,6,Install (App),171,169
5,True,False,2024,6,Install (App),226,226
6,False,False,2024,6,Sign-in (App),507488,498910
7,True,True,2024,6,Sign-in (App),41258,41255
8,False,True,2024,6,Sign-in (App),6542,6085
9,True,False,2024,6,Sign-in (App),1958,1972


In [ ]:
df.query("Event_Category=='Sign-in (App)'").abid.sum()

557333

In [ ]:
query = """ 
SELECT
  i.Is_First_Event_per_User_ID,
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, Event_Category, 
  count(distinct Airbridge_Device_ID) as abid, count(distinct User_ID) as wkid
 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30"
  and Event_Category in ("Install (App)", "Sign-in (App)")
 group by year, month, Event_Category, i.Is_First_Event_per_User_ID
"""

def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,Is_First_Event_per_User_ID,year,month,Event_Category,abid,wkid
0,<NA>,2024,6,Install (App),118049,0
1,False,2024,6,Install (App),789,745
2,True,2024,6,Install (App),967,967
3,False,2024,6,Sign-in (App),510100,500236
4,True,2024,6,Sign-in (App),43188,43227
5,<NA>,2024,6,Sign-in (App),87,0


In [ ]:
df.query("Event_Category=='Sign-in (App)'").abid.sum(),df.query("Event_Category=='Sign-in (App)'").wkid.sum()

(556797, 547602)

In [24]:
query = """ 
SELECT
  Is_First_Event_per_User_ID,
  EXTRACT(month from Event_Date) as month, 
  Event_Category, 
  COUNT(DISTINCT Airbridge_Device_ID) as abid, 
  COUNT(DISTINCT User_ID) as wkid
FROM `ballosodeuk.airbridge_lake.app_2024`
WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30"
  AND Event_Category in ("Install (App)", "Sign-in (App)")
GROUP BY month, Event_Category, Is_First_Event_per_User_ID
"""
estimate_query_cost(client,query)
df = run_query(client,query).to_dataframe()
df

예상 쿼리 용량: 105889739.51 KB, (100.98 GB)


,Is_First_Event_per_User_ID,month,Event_Category,abid,wkid
0,True,6,Install (App),968,968
1,False,6,Sign-in (App),510100,500389
2,<NA>,6,Install (App),118055,0
3,True,6,Sign-in (App),98220,99087
4,False,6,Install (App),789,747
5,<NA>,6,Sign-in (App),318,0


In [35]:
query = """ 
SELECT
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, count(Event_Category), count(distinct Airbridge_Device_ID)

 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30" and Event_Category = "Sign-in (App)" and i.Is_First_Event_per_User_ID = True
 group by year, month
"""

def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,year,month,f0_,f1_
0,2024,6,43658,43188


In [38]:
query = """ 
SELECT
  extract(year from Event_Date) as year, extract(month from Event_Date) as month, d.Label , count(*), count(distinct Airbridge_Device_ID), sum(d.Value)

 FROM `ballosodeuk.airbridge_mart.app_keyfeatures` , UNNEST(e) as e,  UNNEST(d) as d
 WHERE Event_Date BETWEEN  "2024-06-01" and "2024-06-30" and Event_Category = "view_get__reward_done (App)"
 group by year, month, d.Label
"""

def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

df = run_query(client,query).to_dataframe()
df

,year,month,Label,f0_,f1_,f2_
0,2024,6,소득받기,4271851,221148,427874344.0
1,2024,6,가위바위보,970557,65163,-12456660.0
2,2024,6,보너스봉투_메인,6536880,51210,103800513.0
3,2024,6,홈_챌린지리스트,1930659,155890,19306590.0
4,2024,6,오퍼월,5221205,78567,21528355.0
5,2024,6,친구초대입력,2351,2349,2351000.0
6,2024,6,챌린지,28988,28877,4348200.0
7,2024,6,혜택_리스트,848651,62804,8486510.0
8,2024,6,매일버튼누르기챌린지,9837870,158630,11034798.0
9,2024,6,홈_상단_캐시버튼,781268,205263,7812680.0
